# Results and models

#### Setup and load dataset

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
from src.motion_refiner_4D import Motion_refiner, MAX_NUM_OBJS
from src.config import *
from src.functions import *

mr = Motion_refiner(load_models=True ,traj_n = traj_n, locality_factor=True, clip_only=False)
feature_indices, obj_sim_indices, obj_poses_indices, traj_indices = mr.get_indices()
embedding_indices = mr.embedding_indices

#============================== load dataset ==========================================
X,Y, data = mr.load_dataset(dataset_name, filter_data = True, base_path=data_folder)
X_train, X_test, X_valid, y_train, y_test, y_valid, indices_train, indices_test, indices_val = mr.split_dataset(X, Y, test_size=0.2, val_size=0.1)

test_dataset = tf.data.Dataset.from_tensor_slices((mr.prepare_x(X_test),
                                                    list_to_wp_seq(y_test,d=4),
                                                    X_test[:,embedding_indices])).batch(X_test.shape[0])

g = generator(test_dataset,stop=True,augment=False)
x_t, y_t = next(g)

#==============================0000000000000==========================================
def evaluate_model(model, x_t, y_t):

    print("\nwith next waypoint prediction")
    result_eval = model.evaluate(x_t,y_t)
    print("MSE: ",result_eval)
    print("---------------------------------------------------")
    print("with autoregressive generation:")

    pred = generate(model ,x_t, traj_n=traj_n).numpy()
    result_gen = np.average((y_t - pred[:,1:,:])**2)
    print("MSE: ",result_gen)
    print("Trajectory metrics:")
    metrics, metrics_h = compute_metrics(y_t.numpy()[:,:,:3],pred[:,1:,:3])
    return metrics


loading BERT model... 

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


done
loading CLIP model... done
DEVICE:  cuda
loading dataset:  4D_100k_imgs_sf ...done
raw X: (100000, 953) 	Y: (100000, 160)
filtered X: (96771, 953) 	Y: (96771, 160)
Train X: (67739, 953) 	Y: (67739, 160)
Test  X: (19355, 953) 	Y: (19355, 160)
Val   X: (9677, 953) 	Y: (9677, 160)


### dataset size experiment

In [2]:
from src.TF4D_mult_features import *
model_path = models_folder+"ICRA_TF4D_dataset_size_aug/"

model_names =  ["TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.01-augment:0.h5",
                "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.01-augment:1.h5",
                "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.1-augment:0.h5",
                "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.1-augment:1.h5",
                "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:0.h5",
                "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5"]
models_metrics = {}
for model_name in model_names:

    model_file = model_path+model_name
    param = file_name2dict(model_file,delimiter="-",show=False)
    print("======================================================================================================")
    print("Dataset size: ",str(100.0*float(param["sf"])),"%\t","augmentation: ","ON "if param['augment']==1 else "OFF", "\n")
    model_tag = "size:" + str(100.0*float(param["sf"]))+"_aug:"+ str(param['augment'])

    model = load_model(model_file, delimiter="-")
    metrics = evaluate_model(model, x_t, y_t)
    models_metrics[model_tag] = metrics


Dataset size:  1.0 %	 augmentation:  OFF 

{'num_layers_enc': 1, 'num_layers_dec': 5, 'd_model': 400, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 4, 'num_emb_vec': 4, 'bs': 16, 'dense_n': 512, 'num_dense': 3, 'concat_emb': False, 'features_n': 793, 'optimizer': 'adam', 'norm_layer': True, 'activation': 'tanh', 'loss': 'mse', 'sf': 0.01, 'augment': 0}
loading weights:  /home/azureuser/data/models/ICRA_TF4D_dataset_size_aug/TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.01-augment:0.h5

with next waypoint prediction
605/605 [==============================] - 16s 23ms/step - loss: 0.0011
MSE:  0.001147542498074472
---------------------------------------------------
with autoregressive generation:


100%|██████████| 39/39 [07:24<00:00, 11.39s/it]


MSE:  0.04642006029187642
Trajectory metrics:
pcm:	 nan
dfd:	 0.4855052467987429
area:	 0.11811484325016983
cl:	 nan
dtw:	 9.907811929425844
mae:	 0.14150047427352047
mse:	 0.03781905153733537
Dataset size:  1.0 %	 augmentation:  ON  

{'num_layers_enc': 1, 'num_layers_dec': 5, 'd_model': 400, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 4, 'num_emb_vec': 4, 'bs': 16, 'dense_n': 512, 'num_dense': 3, 'concat_emb': False, 'features_n': 793, 'optimizer': 'adam', 'norm_layer': True, 'activation': 'tanh', 'loss': 'mse', 'sf': 0.01, 'augment': 1}
loading weights:  /home/azureuser/data/models/ICRA_TF4D_dataset_size_aug/TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.01-augment:1.h5

with next waypoint prediction
605/605 [==============================] - 16s 22ms/step - loss: 9.0490e-04
MSE:  0.0009083

100%|██████████| 39/39 [07:18<00:00, 11.25s/it]


MSE:  0.04445634728509167
Trajectory metrics:
pcm:	 nan
dfd:	 0.4822911294821316
area:	 0.12010433962268263
cl:	 nan
dtw:	 9.707516932851473
mae:	 0.14041032651210336
mse:	 0.03730230580838904
Dataset size:  10.0 %	 augmentation:  OFF 

{'num_layers_enc': 1, 'num_layers_dec': 5, 'd_model': 400, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 4, 'num_emb_vec': 4, 'bs': 16, 'dense_n': 512, 'num_dense': 3, 'concat_emb': False, 'features_n': 793, 'optimizer': 'adam', 'norm_layer': True, 'activation': 'tanh', 'loss': 'mse', 'sf': 0.1, 'augment': 0}
loading weights:  /home/azureuser/data/models/ICRA_TF4D_dataset_size_aug/TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:0.1-augment:0.h5

with next waypoint prediction
605/605 [==============================] - 16s 22ms/step - loss: 1.3217e-04
MSE:  0.00013212

...

MSE:  0.002223579699245976
Trajectory metrics:
pcm:	 nan
dfd:	 0.05313825742322576
area:	 0.02192462795033975
cl:	 nan
dtw:	 1.2487861897980428
mae:	 0.016873071055151227
mse:	 0.0008171283799243248
Dataset size:  100.0 %	 augmentation:  OFF 

{'num_layers_enc': 1, 'num_layers_dec': 5, 'd_model': 400, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 4, 'num_emb_vec': 4, 'bs': 16, 'dense_n': 512, 'num_dense': 3, 'concat_emb': False, 'features_n': 793, 'optimizer': 'adam', 'norm_layer': True, 'activation': 'tanh', 'loss': 'mse', 'sf': 1.0, 'augment': 0}
loading weights:  /home/azureuser/data/models/ICRA_TF4D_dataset_size_aug/TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:0.h5

with next waypoint prediction
605/605 [==============================] - 16s 22ms/step - loss: 9.7226e-05
MSE:  9.6

100%|██████████| 39/39 [07:16<00:00, 11.19s/it]


MSE:  0.002402131624049286
Trajectory metrics:
pcm:	 nan
dfd:	 0.057981619702562145
area:	 0.02590039823868204
cl:	 nan
dtw:	 1.3980032322981384
mae:	 0.019107776578979774
mse:	 0.0010874291330305122
Dataset size:  100.0 %	 augmentation:  ON  

{'num_layers_enc': 1, 'num_layers_dec': 5, 'd_model': 400, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 4, 'num_emb_vec': 4, 'bs': 16, 'dense_n': 512, 'num_dense': 3, 'concat_emb': False, 'features_n': 793, 'optimizer': 'adam', 'norm_layer': True, 'activation': 'tanh', 'loss': 'mse', 'sf': 1.0, 'augment': 1}
loading weights:  /home/azureuser/data/models/ICRA_TF4D_dataset_size_aug/TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5

with next waypoint prediction
605/605 [==============================] - 15s 22ms/step - loss: 9.7997e-05
MSE:  9.

100%|██████████| 39/39 [07:18<00:00, 11.24s/it]


MSE:  0.002434250811546625
Trajectory metrics:
pcm:	 nan
dfd:	 0.06272383677183252
area:	 0.027304498562373426
cl:	 nan
dtw:	 1.4905868040355599
mae:	 0.02052867658111762
mse:	 0.0012096910789396484


### Num of encoders, decoder and model depth

In [ ]:
from src.TF4D_mult_features import *
model_path = models_folder+"ICRA_TF4D_enc_dec_depth/"

model_names =  ["TF-num_layers_enc:1-num_layers_dec:3-d_model:256-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5",
                "TF-num_layers_enc:1-num_layers_dec:3-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5",
                "TF-num_layers_enc:1-num_layers_dec:5-d_model:256-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5",
                "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5",
                "TF-num_layers_enc:2-num_layers_dec:3-d_model:256-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5",
                "TF-num_layers_enc:2-num_layers_dec:3-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5",
                "TF-num_layers_enc:2-num_layers_dec:5-d_model:256-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5",
                "TF-num_layers_enc:2-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:4-bs:16-dense_n:512-num_dense:3-concat_emb:False-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse-sf:1.0-augment:1.h5"]
models_metrics_enc_dec = {}
for model_name in model_names:

    model_file = model_path+model_name
    param = file_name2dict(model_file,delimiter="-",show=False)
    print("======================================================================================================")
    print("num Encoders: ",param["num_layers_enc"],"\tnum Decoders: ",param["num_layers_dec"],"\tDepth: ",param['d_model'])
    model_tag = "enc:"+str(param["num_layers_enc"])+"-dec:"+str(param["num_layers_dec"])+"-d"+str(param['d_model'])

    model = load_model(model_file, delimiter="-")
    metrics = evaluate_model(model, x_t, y_t)
    models_metrics[model_tag] = metrics

### clip only

1,3,256
1,3,400
1,5,256
1,5,400
2,3,256
2,3,400
2,5,256
2,5,400

### Forces interactions

In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
from src.motion_refiner_4D import Motion_refiner, MAX_NUM_OBJS
from src.config import *
from src.functions import *

mr = Motion_refiner(load_models=True ,traj_n = traj_n, locality_factor=True, clip_only=False)
feature_indices, obj_sim_indices, obj_poses_indices, traj_indices = mr.get_indices()
embedding_indices = mr.embedding_indices

#============================== load dataset ==========================================
X,Y, data = mr.load_dataset("forces_only_f", filter_data = True, base_path=data_folder)
X_train, X_test, X_valid, y_train, y_test, y_valid, indices_train, indices_test, indices_val = mr.split_dataset(X, Y, test_size=0.2, val_size=0.1)


def prepare_x(x):
    objs = pad_array(list_to_wp_seq(x[:,obj_poses_indices],d=3),4,axis=-1) # no speed
    trajs = list_to_wp_seq(x[:,traj_indices],d=4)
    #   return np.concatenate([objs,trajs],axis = 1)
    return trajs[:,:-1,:]

test_dataset = tf.data.Dataset.from_tensor_slices((prepare_x(X_test),
                                                    list_to_wp_seq(y_test,d=4),
                                                    X_test[:,embedding_indices])).batch(X_test.shape[0])

g = generator(test_dataset,stop=True,augment=False)
x_t, y_t = next(g)



loading BERT model... 

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


done
loading CLIP model... done
DEVICE:  cuda
loading dataset:  forces_only_f ...done
raw X: (10000, 953) 	Y: (10000, 160)
filtered X: (10000, 953) 	Y: (10000, 160)
Train X: (7000, 953) 	Y: (7000, 160)
Test  X: (2000, 953) 	Y: (2000, 160)
Val   X: (1000, 953) 	Y: (1000, 160)
{'num_layers_enc': 1, 'num_layers_dec': 5, 'd_model': 400, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 4, 'num_emb_vec': 16, 'bs': 16, 'dense_n': 512, 'num_dense': 3, 'concat_emb': True, 'features_n': 793, 'optimizer': 'adam', 'norm_layer': True, 'activation': 'tanh', 'loss': 'mse'}
loading weights:  /home/azureuser/data/models/forces_onl/TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:16-bs:16-dense_n:512-num_dense:3-concat_emb:True-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse.h5


In [5]:
from src.TF4D_decoder_only import *

model_path = models_folder+"forces_onl/"
model_name = "TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:16-bs:16-dense_n:512-num_dense:3-concat_emb:True-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse.h5"

model_file = model_path+model_name
model = load_model(model_file, delimiter="-")

# metrics = evaluate_model(model, x_t, y_t)
# models_metrics[model_tag] = metrics

{'num_layers_enc': 1, 'num_layers_dec': 5, 'd_model': 400, 'dff': 512, 'num_heads': 8, 'dropout_rate': 0.1, 'wp_d': 4, 'num_emb_vec': 16, 'bs': 16, 'dense_n': 512, 'num_dense': 3, 'concat_emb': True, 'features_n': 793, 'optimizer': 'adam', 'norm_layer': True, 'activation': 'tanh', 'loss': 'mse'}
loading weights:  /home/azureuser/data/models/forces_onl/TF-num_layers_enc:1-num_layers_dec:5-d_model:400-dff:512-num_heads:8-dropout_rate:0.1-wp_d:4-num_emb_vec:16-bs:16-dense_n:512-num_dense:3-concat_emb:True-features_n:793-optimizer:adam-norm_layer:True-activation:tanh-loss:mse.h5


In [6]:
pred = model.predict(x_t)

2022-09-12 22:48:38.680329: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-09-12 22:48:38.680738: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2593990000 Hz
2022-09-12 22:48:40.706933: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


In [13]:
pred_d = np.array(data)[indices_test]
for i,d in enumerate(pred_d):
    d["forces"] = pred[i]
# plt.close('all')
# indices = np.random.choice(range(len(indices_test)), 6)
# pred_t = np.transpose(pred[:,:,:2],[0,2,1])

show_data4D(pred_d,plot_output=False,image_loader=mr.image_loader, color_traj=False, change_img_base=["/home/mirmi/Arthur/dataset/","/home/tum/data/image_dataset/"])
# show_data4D(data_sample,plot_output=False)#,image_loader=mr.image_loader)

TypeError: show_data4D() got an unexpected keyword argument 'plot_output'

# best model